<a href="https://colab.research.google.com/github/Meow-nah-ngo/LPR/blob/main/LPR_Kwan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  **YoloV8**

In [26]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 901.3/901.3 kB 20.2 MB/s eta 0:00:00


In [27]:
from ultralytics import YOLO
# Load a model
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 118MB/s]


#Easyocr

In [28]:
pip install easyocr


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 912.2/912.2 kB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.8/286.8 kB 26.4 MB/s eta 0:00:00


#Other pip

In [29]:
pip install opencv-python

In [30]:
pip install fuzzywuzzy


In [31]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="wGem50Rjvyol0TlOkRk7")
project = rf.workspace("p-opz7u").project("l-abonw")
version = project.version(2)
dataset = version.download("yolov8")
#Dataset from roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.4 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to l-2 in yolov8:: 100%|██████████| 3982/3982 [00:00<00:00, 9937.61it/s]


In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="wGem50Rjvyol0TlOkRk7")
project = rf.workspace("ai-888").project("license-plate-y3o6h")
version = project.version(1)
dataset = version.download("yolov8")
#Dataset from roboflow

loading Roboflow workspace...
loading Roboflow project...


#Model

In [ ]:
import easyocr
import os
import cv2
import re
from collections import defaultdict

# EasyOCR
reader = easyocr.Reader(['th'], gpu=True)

# ที่เก็บรูปภาพ
folder_paths = ['l-2/train/images', 'license-plate-1/train/images']
results = defaultdict(list)  # เก็บผลลัพธ์

# ฟังก์ชันกรองข้อมูลที่ไม่จำเป็น
def clean_text(text):
    return text.strip() if len(text.strip()) > 1 and text.strip() not in ['.', '!', '#', 'ๆ', '…'] else None

# ฟังก์ชันตรวจจับเลขทะเบียนรถ
def is_license_plate(text):
    return bool(re.match(r'^[0-9]{1,2}\s?[A-Za-zก-ฮ]{1,2}\s?[0-9]{1,4}$', text))

# ฟังก์ชันตรวจจับจังหวัด
def is_province(text):
    provinces = [
        "ขอนแก่น", "จันทบุรี", "ฉะเชิงเทรา", "ชลบุรี", "ชัยนาท", "ชัยภูมิ", "ชุมพร","กรุงเทพมหานคร",
        "เชียงใหม่", "เชียงราย", "ตรัง", "ตราด", "ตาก", "นครนายก", "นครปฐม", "นครพนม", "นครราชสีมา", "นครศรีธรรมราช",
        "นครสวรรค์", "นราธิวาส", "น่าน", "บึงกาฬ", "บุรีรัมย์", "ปทุมธานี", "ประจวบคีรีขันธ์", "ปราจีนบุรี", "ปัตตานี",
        "พะเยา", "พังงา", "พัทลุง", "พิจิตร", "พิษณุโลก", "เพชรบูรณ์", "เพชรบุรี", "ภูเก็ต", "มุกดาหาร",
        "ยโสธร", "ระยอง", "ราชบุรี", "ลำปาง", "ลำพูน", "เลย", "ศรีสะเกษ", "สกลนคร", "สมุทรปราการ", "สมุทรสงคราม",
        "สมุทรสาคร", "สระแก้ว", "สระบุรี", "สิงห์บุรี", "สุพรรณบุรี", "สุราษฎร์ธานี", "สุรินทร์", "สงขลา", "สุโขทัย",
        "สตูล", "อำนาจเจริญ", "อุดรธานี", "อุตรดิตถ์", "อุทัยธานี", "อ่างทอง", "อุบลราชธานี", "สมุทรปราการ",
        "ปทุมธานี", "นราธิวาส", "ประจวบคีรีขันธ์", "ราชบุรี", "กระบี่", "มหาสารคาม", "หนองคาย", "สระแก้ว", "ชัยภูมิ"
    ]

    return any(province.lower() in text.lower() for province in provinces)

# วนลูป check แต่ละรูป
for folder_path in folder_paths:
    for filename in os.listdir(folder_path):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            image_path = os.path.join(folder_path, filename)

            # โหลดภาพ
            image = cv2.imread(image_path)

            # แปลงภาพเป็นขาวดำ
            gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            # ปรับคอนทราสต์
            adjusted_image = cv2.convertScaleAbs(gray_image, alpha=1.5, beta=0)

            # ใช้ EasyOCR กับภาพที่ปรับแต่งแล้ว
            result = reader.readtext(adjusted_image, contrast_ths=0.05, text_threshold=0.4)
            results[filename] = result

            # แสดงผลลัพธ์ OCR ก่อนกรอง
            print(f"\n--- Results for {filename} ---")
            for text in result:
                print(f"Detected text: {text[1]}")

            # กรองข้อความที่ไม่ต้องการ
            cleaned_result = [x[1].strip() for x in result if clean_text(x[1])]
            results[filename] = cleaned_result

# จัดระเบียบข้อมูล ที่อ่านได้ (ยังไม่สามารถตรวจจับจังหวัดอื่นได้)
organized_data = []

for filename, result in results.items():
    license_plate = None
    province = None

    for text in result:
        if is_license_plate(text):
            license_plate = text
        elif is_province(text):
            province = text

    if license_plate and province:
        organized_data.append({'filename': filename, 'license_plate': license_plate, 'province': province})

# แสดงผลข้อมูลที่จัดระเบียบแล้วในรูปแบบที่เข้าใจง่าย
print("\n--- Organized Data ---")
if organized_data:
    for data in organized_data:
        print(f"File: {data['filename']}")
        print(f"  License Plate: {data['license_plate']}")
        print(f"  Province: {data['province']}")
else:
    print("No valid license plates and provinces detected.")


เอาต์พุตของการสตรีมมีการตัดเหลือเพียง 5000 บรรทัดสุดท้าย

--- Results for 0-69160842_2248848585242081_5443583130252345344_n_jpg.rf.c2368254eb8ada4217cd8f1b9161b5d4.jpg ---
Detected text: ขธ 6708
Detected text: ชลบุรี

--- Results for 0-260789832_4371581889635396_4862302881056638544_n_jpg.rf.5693cb8485d48c4bbb4fe25d5e73c05f.jpg ---
Detected text: ฆษ 7778
Detected text: กรุงเทพมหานคร

--- Results for 0-150034776_3540459356080991_8439398441610037356_n_jpg.rf.7726fa292655415460087868ecedd601.jpg ---
Detected text: ขค.
Detected text: อี
Detected text: ชอบรี

--- Results for 0-44171286_1762140917246186_6359378279266254848_n_jpg.rf.bc892764659bd4cdbfb7f0fefcac248d.jpg ---
Detected text: 6กผ8666
Detected text: กรุงเทพมทานคร

--- Results for 0-260227713_4371581879635397_1802514461314083682_n_jpg.rf.d40ac4ecd5d2ab57393b99a25c99c357.jpg ---
Detected text: ฆษ 7778
Detected text: กรุงเทพมหานคร

--- Results for 0-280030712_4877306849062895_888410037043401200_n_jpg.rf.1c55d32b25335e68b8a2b100facf9767

In [ ]:
!pip install gspread oauth2client
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [1]:
from flask import Flask, request, jsonify
import cv2
import easyocr
import re
import numpy as np
import requests # เพิ่มไลบรารี requests เพื่อส่งข้อมูลไป Arduino
app = Flask(__name__)
# EasyOCR setup
reader = easyocr.Reader(['th'], gpu=True)
# ฟังก์ชันกรองข้อมูลที่ไม่จำเป็น
def clean_text(text):
    return text.strip() if len(text.strip()) > 1 and text.strip() not in ['.', '!', '#', 'ๆ', '…'] else None
# ฟังก์ชันตรวจจับเลขทะเบียนรถ
def is_license_plate(text):
    return bool(re.match(r'^[0-9]{1,2}\s?[A-Za-zก-ฮ]{1,2}\s?[0-9]{1,4}$', text))
# ฟังก์ชันตรวจจับจังหวัด
def is_province(text):
    provinces = [
        "ขอนแก่น", "จันทบุรี", "ฉะเชิงเทรา", "ชลบุรี", "ชัยนาท", "ชัยภูมิ", "ชุมพร", "กรุงเทพมหานคร",
        "เชียงใหม่", "เชียงราย", "ตรัง", "ตราด", "ตาก", "นครนายก", "นครปฐม", "นครพนม", "นครราชสีมา", "นครศรีธรรมราช",
        "นครสวรรค์", "นราธิวาส", "น่าน", "บึงกาฬ", "บุรีรัมย์", "ปทุมธานี", "ประจวบคีรีขันธ์", "ปราจีนบุรี", "ปัตตานี",
        "พะเยา", "พังงา", "พัทลุง", "พิจิตร", "พิษณุโลก", "เพชรบูรณ์", "เพชรบุรี", "ภูเก็ต", "มุกดาหาร",
        "ยโสธร", "ระยอง", "ราชบุรี", "ลำปาง", "ลำพูน", "เลย", "ศรีสะเกษ", "สกลนคร", "สมุทรปราการ", "สมุทรสงคราม",
        "สมุทรสาคร", "สระแก้ว", "สระบุรี", "สิงห์บุรี", "สุพรรณบุรี", "สุราษฎร์ธานี", "สุรินทร์", "สงขลา", "สุโขทัย",
        "สตูล", "อำนาจเจริญ", "อุดรธานี", "อุตรดิตถ์", "อุทัยธานี", "อ่างทอง", "อุบลราชธานี", "สมุทรปราการ",
        "ปทุมธานี", "นราธิวาส", "ประจวบคีรีขันธ์", "ราชบุรี", "กระบี่", "มหาสารคาม", "หนองคาย", "สระแก้ว", "ชัยภูมิ"
    ]
    return any(province.lower() in text.lower() for province in provinces)
# ฟังก์ชันส่งข้อมูลไปยัง Arduino API
def send_to_arduino(license_plate, province, image_link):
    arduino_url = "http://arduino.local/api" # URL ของ Arduino ที่รับข้อมูล
    data = {
        'license_plate': license_plate,
        'province': province,
        'image_link': image_link
    }
    try:
        response = requests.post(arduino_url, json=data)
        if response.status_code == 200:
            return "Data sent to Arduino successfully!"
        else:
            return f"Failed to send data to Arduino, status code: {response.status_code}"
    except Exception as e:
        return f"Error sending data to Arduino: {e}"
@app.route('/process_image', methods=['POST'])
def process_image():
    # รับข้อมูลภาพจาก request
    if 'image' not in request.files:
        return jsonify({"error": "No image provided"}), 400
    file = request.files['image']
    image = np.frombuffer(file.read(), np.uint8)
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    # แปลงภาพเป็นขาวดำ
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # ปรับคอนทราสต์
    adjusted_image = cv2.convertScaleAbs(gray_image, alpha=1.5, beta=0)
    # ใช้ EasyOCR กับภาพที่ปรับแต่งแล้ว
    result = reader.readtext(adjusted_image, contrast_ths=0.05, text_threshold=0.4)
    # กรองข้อความที่ไม่ต้องการ
    cleaned_result = [x[1].strip() for x in result if clean_text(x[1])]
    # ตรวจจับทะเบียนรถและจังหวัด
    license_plate = None
    province = None
    image_link = "http://example.com/image.jpg" # ตัวอย่างลิงก์ภาพ
    for text in cleaned_result:
        if is_license_plate(text):
            license_plate = text
        elif is_province(text):
            province = text
    # สร้างผลลัพธ์ที่ต้องการ
    response = {"license_plate": license_plate, "province": province}
    # ส่งข้อมูลไปยัง Arduino API
    if license_plate and province:
        arduino_response = send_to_arduino(license_plate, province, image_link)
        print(arduino_response)
    return jsonify(response)
if __name__ == '__main__':
    app.run(host="0.0.0.0", port=8080)

ModuleNotFoundError: No module named 'easyocr'